In [ ]:
pip install langchain langchain_community sentence-transformers chromadb faiss-cpu langchain_groq langchain_tavily arxiv

In [ ]:
pip install pymupdf

In [ ]:
pip install ragas

In [4]:
import os
from google.colab import userdata

GROQ_API_KEY = userdata.get('GROQ_API_KEY')
os.environ['GROQ_API_KEY'] = GROQ_API_KEY

In [17]:
from langchain.document_loaders import ArxivLoader

base_docs = ArxivLoader(query='Retrieval Augmented Generation', load_max_docs=5, use_pdf=False).load()
len(base_docs)

3

In [18]:
for doc in base_docs:
  print(doc.metadata)

{'Published': '2024-10-30', 'Title': 'R^2AG: Incorporating Retrieval Information into Retrieval Augmented Generation', 'Authors': 'Fuda Ye, Shuangyin Li, Yongqi Zhang, Lei Chen', 'Summary': "Retrieval augmented generation (RAG) has been applied in many scenarios to\naugment large language models (LLMs) with external documents provided by\nretrievers. However, a semantic gap exists between LLMs and retrievers due to\ndifferences in their training objectives and architectures. This misalignment\nforces LLMs to passively accept the documents provided by the retrievers,\nleading to incomprehension in the generation process, where the LLMs are\nburdened with the task of distinguishing these documents using their inherent\nknowledge. This paper proposes R$^2$AG, a novel enhanced RAG framework to fill\nthis gap by incorporating Retrieval information into Retrieval Augmented\nGeneration. Specifically, R$^2$AG utilizes the nuanced features from the\nretrievers and employs a R$^2$-Former to capt

In [19]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=0)
splits = text_splitter.split_documents(base_docs)
len(splits)

321

In [21]:
from langchain.embeddings import HuggingFaceEmbeddings
embedding = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

/tmp/ipython-input-21-3802521539.py:2: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:104: UserWarning: 
Error while fetching `HF_TOKEN` secret value from your vault: 'Requesting secret HF_TOKEN timed out. Secrets can only be fetched when running from the Colab UI.'.
You are not authenticated with the Hugging Face Hub in this notebook.
If the error persists, please let us know by opening an issue on GitHub (https://github.com/huggingface/huggingface_hub/issues/new).
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [22]:
from langchain.vectorstores import Chroma
vector_store = Chroma.from_documents(splits, embedding)

In [23]:
print(max([len(chunk.page_content) for chunk in splits]))

499


In [26]:
from langchain_groq import ChatGroq
from langchain.chains import RetrievalQA

llm = ChatGroq(model="qwen/qwen3-32b")

retriever = vector_store.as_retriever(search_kwargs={"k": 3})

qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retriever,
    return_source_documents=True,
)

In [29]:
query = "what is RAG?"
response = qa_chain.invoke(query)

print(response["result"])

<think>
Okay, the user is asking, "what is RAG?" Let me start by recalling what I know about RAG. From the provided context, there's mention of RAG being enhanced with methods that improve performance by 30% over a baseline. The text also discusses challenges in RAG like sensitivity to retrieval results and a semantic gap between retrievers and LLMs.

RAG stands for Retrieval-Augmented Generation. It's a framework that combines retrieval from a corpus of documents with a language model to generate answers. The basic idea is that the model retrieves relevant documents first and then uses them to generate a response. This approach helps the model provide more accurate and up-to-date information by leveraging external data sources.

In the context provided, there are sections discussing related work and enhanced RAG methods. The standard RAG approach uses simple text concatenation, which might lead to issues like the model not fully understanding why certain documents are retrieved. Enhan

In [31]:
from langchain.output_parsers import ResponseSchema
from langchain.output_parsers import StructuredOutputParser

question_schema = ResponseSchema(
    name="question",
    description="a question about the context."
)

question_response_schemas = [
    question_schema,
]

In [32]:
question_output_parser = StructuredOutputParser.from_response_schemas(question_response_schemas)
format_instructions = question_output_parser.get_format_instructions()

In [47]:
from langchain.prompts import ChatPromptTemplate

qa_template = """\
You are a University Professor creating a test for advanced students. For each context, create a question that is specific to the context. Avoid creating generic or general questions.

question: a question about the context.

Format the output as JSON with the following keys:
question

context: {context}

Produce ONLY the JSON and nothing else. Do not include any introductory or concluding remarks, or any other characters before or after the JSON.
"""

prompt_template = ChatPromptTemplate.from_template(template=qa_template)

In [38]:
messages = prompt_template.format_messages(
    context=splits[0],
    format_instructions=format_instructions
)

In [49]:
import json
import re

response = llm.invoke(messages)
# Extract the JSON string from the response content
# This pattern looks for a string starting with { and ending with } potentially spanning multiple lines
match = re.search(r"\{.*\}", response.content, re.DOTALL)

if match:
    json_string = match.group(0)
    try:
        # Attempt to parse the extracted string
        output_dict = json.loads(json_string)
        print(output_dict)
    except json.JSONDecodeError as e:
        print(f"Error decoding JSON after extraction: {e}")
        print(f"Problematic JSON string: {json_string}")
        # Fallback or error handling if the extracted string is still not valid JSON
        raise e # Re-raise the exception if it still fails after extraction

else:
    print("No JSON object found in the response.")
    print(f"Full response content: {response.content}")
    raise ValueError("Could not find JSON in the response")

{'question': 'How does the R²-Former in R²AG address the semantic gap between large language models (LLMs) and retrievers, and what specific architectural features enable it to capture retrieval information effectively?', 'context': {'page_content': 'R2AG: Incorporating Retrieval Information into Retrieval Augmented Generation\n... the semantic gap exists between LLMs and retrievers due to differences in their training objectives and architectures. This misalignment forces LLMs to passively accept the documents provided by the retrievers, leading to incomprehension in the generation process... R²AG utilizes the nuanced features from the retrievers and employs a R²-Former to capture retrieval information.'}}


In [52]:
for k, v in output_dict.items():
  print(k)
  print(v)

question
How does the R²-Former in R²AG address the semantic gap between large language models (LLMs) and retrievers, and what specific architectural features enable it to capture retrieval information effectively?
context
{'page_content': 'R2AG: Incorporating Retrieval Information into Retrieval Augmented Generation\n... the semantic gap exists between LLMs and retrievers due to differences in their training objectives and architectures. This misalignment forces LLMs to passively accept the documents provided by the retrievers, leading to incomprehension in the generation process... R²AG utilizes the nuanced features from the retrievers and employs a R²-Former to capture retrieval information.'}


In [55]:
from tqdm import tqdm
import json
import re

qac_triples = []

for text in tqdm(splits[:10]):
  messages = prompt_template.format_messages(
      context=text,
      format_instructions=format_instructions
  )
  response = llm.invoke(messages)

  # Extract the JSON string from the response content
  match = re.search(r"\{.*\}", response.content, re.DOTALL)

  if match:
    json_string = match.group(0)
    try:
      # Attempt to parse the extracted string
      output_dict = json.loads(json_string)
      output_dict["context"] = text
      qac_triples.append(output_dict)
    except json.JSONDecodeError as e:
      print(f"Error decoding JSON for text chunk: {text.page_content[:50]}...")
      print(f"Error: {e}")
      print(f"Problematic JSON string: {json_string}")
      continue # Skip this chunk if JSON parsing fails

  else:
    print(f"No JSON object found in response for text chunk: {text.page_content[:50]}...")
    print(f"Full response content: {response.content}")
    continue # Skip this chunk if no JSON is found

 70%|███████   | 7/10 [00:17<00:11,  3.76s/it]

Error decoding JSON for text chunk: trievers and LLMs due to their vastly different tr...
Error: Expecting ',' delimiter: line 1 column 724 (char 723)
Problematic JSON string: {"question": "According to the summary, how does the R²AG framework specifically bridge the semantic gap between retrievers and LLMs in low-source scenarios, and what are the two key components involved in this process?","context": {"page_content": "trievers and LLMs due to their vastly different train-\ning objectives and architectures (BehnamGhader\net al., 2022). Specifically, retrievers, typically en-\ncoder architecture, are designed to retrieve the most\nrelevant documents for a query (Zhu et al., 2023b).\nConversely, LLMs, generally decoder architecture,\nare expected to answer questions based on their\ninherent knowledge or given documents. How-\never, the interaction between retrievers and LLMs' metadata": {"Published": "2024-10-30", "Title": "R^2AG: Incorporating Retrieval Information into Retrieval Aug

100%|██████████| 10/10 [00:58<00:00,  5.85s/it]


In [63]:
qac_triples[8]

{'question': "What specific components does R²AG introduce to integrate retrieval information into the LLMs' generation process, and how do they address the semantic gap between retrievers and LLMs caused by differences in training objectives and architectures?",
 'context': Document(metadata={'Published': '2024-10-30', 'Title': 'R^2AG: Incorporating Retrieval Information into Retrieval Augmented Generation', 'Authors': 'Fuda Ye, Shuangyin Li, Yongqi Zhang, Lei Chen', 'Summary': "Retrieval augmented generation (RAG) has been applied in many scenarios to\naugment large language models (LLMs) with external documents provided by\nretrievers. However, a semantic gap exists between LLMs and retrievers due to\ndifferences in their training objectives and architectures. This misalignment\nforces LLMs to passively accept the documents provided by the retrievers,\nleading to incomprehension in the generation process, where the LLMs are\nburdened with the task of distinguishing these documents u

In [66]:
from langchain.output_parsers import ResponseSchema
from langchain.output_parsers import StructuredOutputParser
from langchain.prompts import ChatPromptTemplate
import json
import re


answer_schema = ResponseSchema(
    name="answer",
    description="an answer to the question"
)

answer_response_schemas = [
    answer_schema,
]

answer_output_parser = StructuredOutputParser.from_response_schemas(answer_response_schemas)
format_instructions = answer_output_parser.get_format_instructions()

qa_template = """\
You are a University Professor creating a test for advanced students. For each question and context, create an answer.

answer: a answer about the context.

Format the output as JSON with the following keys:
answer

question: {question}
context: {context}

Produce ONLY the JSON and nothing else. Do not include any introductory or concluding remarks, or any other characters before or after the JSON.
"""

prompt_template = ChatPromptTemplate.from_template(template=qa_template)

messages = prompt_template.format_messages(
    context=qac_triples[0]["context"],
    question=qac_triples[0]["question"],
    format_instructions=format_instructions
)

response = llm.invoke(messages)

# Extract the JSON string from the response content
match = re.search(r"\{.*\}", response.content, re.DOTALL)

if match:
    json_string = match.group(0)
    try:
        # Attempt to parse the extracted string
        output_dict = json.loads(json_string)
    except json.JSONDecodeError as e:
        print(f"Error decoding JSON after extraction: {e}")
        print(f"Problematic JSON string: {json_string}")
        # Fallback or error handling if the extracted string is still not valid JSON
        raise e # Re-raise the exception if it still fails after extraction

else:
    print("No JSON object found in the response.")
    print(f"Full response content: {response.content}")
    raise ValueError("Could not find JSON in the response")

In [67]:
for k, v in output_dict.items():
  print(k)
  print(v)

answer
The retrieval-aware prompting strategy in R²AG integrates retrieval information by designing prompts that explicitly incorporate nuanced features from retrievers. These prompts guide the LLM to leverage retrieval data during generation, enabling the model to align its outputs with the retrieved documents. This approach addresses the semantic gap between LLMs and retrievers by providing structured retrieval cues, allowing the LLM to focus on generating coherent, contextually relevant responses anchored in the retrieved information, rather than passively relying on its own knowledge. The strategy is implemented without retraining frozen LLMs or retrievers, ensuring efficiency in low-source scenarios.


In [69]:
from tqdm import tqdm
import json
import re

for triple in tqdm(qac_triples):
  messages = prompt_template.format_messages(
      context=triple["context"],
      question=triple["question"],
      format_instructions=format_instructions
  )
  response = llm.invoke(messages)

  # Extract the JSON string from the response content
  match = re.search(r"\{.*\}", response.content, re.DOTALL)

  if match:
    json_string = match.group(0)
    try:
      # Attempt to parse the extracted string
      output_dict = json.loads(json_string)
      triple["answer"] = output_dict.get("answer", "No answer generated.") # Use .get() with a default value
    except json.JSONDecodeError as e:
      print(f"Error decoding JSON for question: {triple['question'][:50]}...")
      print(f"Error: {e}")
      print(f"Problematic JSON string: {json_string}")
      triple["answer"] = "Error generating answer." # Assign an error message
      continue # Continue with the next triple

  else:
    print(f"No JSON object found in response for question: {triple['question'][:50]}...")
    print(f"Full response content: {response.content}")
    triple["answer"] = "No answer generated." # Assign a message indicating no JSON was found
    continue # Continue with the next triple

100%|██████████| 9/9 [00:37<00:00,  4.11s/it]


In [72]:
qac_triples[5]

{'question': 'Explain how R²AG addresses the semantic gap between retrievers and LLMs, focusing on the role of the R²-Former and retrieval-aware prompting strategy as described in the paper.',
 'context': Document(metadata={'Published': '2024-10-30', 'Title': 'R^2AG: Incorporating Retrieval Information into Retrieval Augmented Generation', 'Authors': 'Fuda Ye, Shuangyin Li, Yongqi Zhang, Lei Chen', 'Summary': "Retrieval augmented generation (RAG) has been applied in many scenarios to\naugment large language models (LLMs) with external documents provided by\nretrievers. However, a semantic gap exists between LLMs and retrievers due to\ndifferences in their training objectives and architectures. This misalignment\nforces LLMs to passively accept the documents provided by the retrievers,\nleading to incomprehension in the generation process, where the LLMs are\nburdened with the task of distinguishing these documents using their inherent\nknowledge. This paper proposes R$^2$AG, a novel en

In [73]:
import pandas as pd
from datasets import Dataset

In [74]:
ground_truth_qac_set = pd.DataFrame(qac_triples)
ground_truth_qac_set["context"] = ground_truth_qac_set["context"].map(lambda x: str(x.page_content))
ground_truth_qac_set = ground_truth_qac_set.rename(columns={"answer" : "ground_truth"})


eval_dataset = Dataset.from_pandas(ground_truth_qac_set)

In [75]:
eval_dataset

Dataset({
    features: ['question', 'context', 'ground_truth'],
    num_rows: 9
})

In [76]:
eval_dataset[0]

{'question': 'In the R²AG framework, how does the retrieval-aware prompting strategy integrate retrieval information into the generation process of large language models (LLMs)?',
 'context': 'R2AG: Incorporating Retrieval Information into Retrieval Augmented\nGeneration\nFuda Ye1, Shuangyin Li1,*, Yongqi Zhang2, Lei Chen2,3\n1School of Computer Science, South China Normal University\n2The Hong Kong University of Science and Technology (Guangzhou)\n3The Hong Kong University of Science and Technology\nfudayip@m.scnu.edu.cn, shuangyinli@scnu.edu.cn, yongqizhang@hkust-gz.edu.cn, leichen@cse.ust.hk\nAbstract\nRetrieval augmented generation (RAG) has\nbeen applied in many scenarios to augment',
 'ground_truth': 'The retrieval-aware prompting strategy in R²AG integrates retrieval information by first extracting nuanced features from retrievers using an R²-Former, which captures the contextual relevance of retrieved documents. These features are then explicitly embedded into the prompts provi

In [77]:
eval_dataset.to_csv("groundtruth_eval_dataset.csv")

Creating CSV from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

13689

In [104]:
from ragas.metrics import answer_relevancy, faithfulness, context_precision, context_recall, answer_correctness, answer_similarity
from ragas import evaluate
from langchain_groq import ChatGroq
from langchain.embeddings import HuggingFaceEmbeddings


def create_ragas_dataset(rag_pipeline, eval_dataset):
  rag_dataset = []
  for row in tqdm(eval_dataset):
    # Changed "question" to "query" as expected by the RetrievalQA chain
    answer = rag_pipeline.invoke({"query" : row["question"]})
    rag_dataset.append(
        {"question" : row["question"],
         "answer" : answer["result"], # Use "result" key as per the output of qa_chain.invoke
         "contexts" : [context.page_content for context in answer["source_documents"]], # Use "source_documents" key as per the output of qa_chain.invoke
         "ground_truths" : [row["ground_truth"]],
         "reference" : row["context"] # Corrected to pass the context string directly
         }
    )
  rag_df = pd.DataFrame(rag_dataset)
  rag_eval_dataset = Dataset.from_pandas(rag_df)
  return rag_eval_dataset

def evaluate_ragas_dataset(ragas_dataset, llm, embeddings):
  result = evaluate(
    ragas_dataset,
    metrics=[
        context_precision,
        faithfulness,
        answer_relevancy,
        context_recall,
        answer_correctness,
        answer_similarity
    ],
    llm=llm, # Pass the LLM
    embeddings=embeddings # Pass the embeddings
  )
  return result

In [88]:
import pandas as pd

ragas_dataset = create_ragas_dataset(qa_chain, eval_dataset)

100%|██████████| 9/9 [01:52<00:00, 12.48s/it]


In [94]:
ragas_dataset[0]

{'question': 'In the R²AG framework, how does the retrieval-aware prompting strategy integrate retrieval information into the generation process of large language models (LLMs)?',
 'answer': '<think>\nOkay, so the user is asking about how the retrieval-aware prompting strategy in the R²AG framework integrates retrieval information into the LLM\'s generation process. Let me start by recalling what I know from the provided context.\n\nFirst, the context mentions that R2AG uses a R2-Former to capture retrieval information and then employs a retrieval-aware prompting strategy. The key part here is the prompting strategy. The description says it\'s designed to insert retrieval information into the LLM\'s input embedding space. \n\nHmm, the user wants to know how this integration works specifically. The context states that the strategy integrates retrieval info without causing information loss or significant complexity. Also, there\'s a mention of using learnable tokens as retrieval informat

In [95]:
ragas_dataset.to_csv("basic_qa_ragas_dataset.csv")

Creating CSV from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

65515

In [102]:
import pandas as pd

basic_qa_ragas_dataset = create_ragas_dataset(qa_chain, eval_dataset)

100%|██████████| 9/9 [01:22<00:00,  9.16s/it]


In [105]:
evalutaion = evaluate_ragas_dataset(basic_qa_ragas_dataset, llm, embedding)

Evaluating:   0%|          | 0/54 [00:00<?, ?it/s]

ERROR:ragas.executor:Exception raised in Job[18]: RateLimitError(Error code: 429 - {'error': {'message': 'Rate limit reached for model `qwen/qwen3-32b` in organization `org_01k0bwwgwgfv9tyrdkfcgwfjt1` service tier `on_demand` on tokens per minute (TPM): Limit 6000, Used 33659, Requested 1306. Please try again in 4m49.651s. Need more tokens? Upgrade to Dev Tier today at https://console.groq.com/settings/billing', 'type': 'tokens', 'code': 'rate_limit_exceeded'}})
ERROR:ragas.executor:Exception raised in Job[22]: RateLimitError(Error code: 429 - {'error': {'message': 'Rate limit reached for model `qwen/qwen3-32b` in organization `org_01k0bwwgwgfv9tyrdkfcgwfjt1` service tier `on_demand` on tokens per minute (TPM): Limit 6000, Used 30929, Requested 1488. Please try again in 4m24.176s. Need more tokens? Upgrade to Dev Tier today at https://console.groq.com/settings/billing', 'type': 'tokens', 'code': 'rate_limit_exceeded'}})
ERROR:ragas.executor:Exception raised in Job[10]: RateLimitError(E

In [108]:
evalutaion

{'context_precision': nan, 'faithfulness': 0.7500, 'answer_relevancy': 0.9345, 'context_recall': 0.6333, 'answer_correctness': 0.3274, 'semantic_similarity': 0.6291}

In [116]:
def create_qa_chain(retriever):
  llm = ChatGroq(model="qwen/qwen3-32b")

  qa_chain = RetrievalQA.from_chain_type(
      llm=llm,
      retriever=retriever,
      return_source_documents=True,
  )
  return qa_chain

In [117]:
from langchain.retrievers import ParentDocumentRetriever
from langchain.storage import InMemoryStore

parent_splitter = RecursiveCharacterTextSplitter(chunk_size=1500)
child_splitter = RecursiveCharacterTextSplitter(chunk_size=200)

vectorstore = Chroma(collection_name="split_parents", embedding_function=embedding)

store = InMemoryStore()

In [118]:
parent_document_retriever = ParentDocumentRetriever(
    vectorstore=vectorstore,
    docstore=store,
    child_splitter=child_splitter,
    parent_splitter=parent_splitter,
)

In [119]:
parent_document_retriever.add_documents(base_docs)

In [120]:
parent_document_retriever_qa_chain = create_qa_chain(parent_document_retriever)

In [124]:
parent_document_retriever_qa_chain.invoke({"query" : "What is RAG?"})['result']

'<think>\nOkay, the user is asking, "What is RAG?" I need to use the provided context to answer this. Let me look through the given text.\n\nFrom the context, RAG is mentioned in several places. The first part talks about Retrieval-augmented Generation, specifically about refining retrieved content using meta-prompting. The second part discusses how RAG combines retrieval with LLMs (Large Language Models) to enhance performance by accessing up-to-date data. It also mentions that RAG has been extended to different domains with modality-specific retrievers like audio, images, etc.\n\nSo, putting this together, RAG is a method that uses retrieval and generation, where retrieved documents are used alongside the query to generate responses. It\'s used to improve the performance of LLMs by providing them with relevant data. The limitations mentioned include sensitivity to retrieval results and a semantic gap between retrievers and LLMs.\n\nI should explain RAG in simple terms, mention its co

In [125]:
pdr_qa_ragas_dataset = create_ragas_dataset(parent_document_retriever_qa_chain, eval_dataset)

100%|██████████| 9/9 [01:29<00:00,  9.97s/it]


In [126]:
pdr_qa_ragas_dataset.to_csv("pdr_qa_ragas_dataset.csv")

Creating CSV from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

80312

In [129]:
pdr_qa_result = evaluate_ragas_dataset(pdr_qa_ragas_dataset, llm, embedding)

Evaluating:   0%|          | 0/54 [00:00<?, ?it/s]

ERROR:ragas.executor:Exception raised in Job[20]: RateLimitError(Error code: 429 - {'error': {'message': 'Rate limit reached for model `qwen/qwen3-32b` in organization `org_01k0bwwgwgfv9tyrdkfcgwfjt1` service tier `on_demand` on tokens per minute (TPM): Limit 6000, Used 37287, Requested 1478. Please try again in 5m27.657s. Need more tokens? Upgrade to Dev Tier today at https://console.groq.com/settings/billing', 'type': 'tokens', 'code': 'rate_limit_exceeded'}})
ERROR:ragas.executor:Exception raised in Job[21]: RateLimitError(Error code: 429 - {'error': {'message': 'Rate limit reached for model `qwen/qwen3-32b` in organization `org_01k0bwwgwgfv9tyrdkfcgwfjt1` service tier `on_demand` on tokens per minute (TPM): Limit 6000, Used 37037, Requested 1893. Please try again in 5m29.306999999s. Need more tokens? Upgrade to Dev Tier today at https://console.groq.com/settings/billing', 'type': 'tokens', 'code': 'rate_limit_exceeded'}})
ERROR:ragas.executor:Exception raised in Job[0]: RateLimitEr

In [130]:
pdr_qa_result

{'context_precision': nan, 'faithfulness': nan, 'answer_relevancy': 0.9135, 'context_recall': 0.6667, 'answer_correctness': nan, 'semantic_similarity': 0.6444}